In [1]:
from array import array
import time
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from numpy import random, arange
from scipy.signal import butter, sosfilt, sosfreqz, lfilter
import datetime
import random
from tkinter import *  
from tkinter.ttk import Progressbar  
from tkinter import ttk  
from tkinter import filedialog  
import sqlalchemy
from scipy import interpolate
import sys
import mysql
import mysql.connector
import pickle
import warnings
import xgboost as xgb
import lightgbm as lgbm
import catboost as catb
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
warnings.simplefilter('ignore')
import pathlib
import os



In [2]:
from forces import vagon_1
from forces import vagon_2
from forces import vagon_3
from forces import vagon_4

In [92]:
line = pd.read_excel('dannye.xlsx', sheet_name='line')
trains = pd.read_excel('dannye.xlsx', sheet_name='trains')
pod_sos = pd.read_excel('dannye.xlsx', sheet_name='pod_sos')

In [93]:
line

,vsp_type,vsp_cstr,vsp_cnd,rad_m,h_mm,i_prm,l_m,Shkol_mm,f_tr
0,1,1,1,10000,0,5,1000,1520,0.25
1,2,0,1,10000,0,0,2500,1525,0.25
2,1,1,1,5000,10,-5,1000,1515,0.35
3,1,0,1,10000,0,0,1000,1530,0.25
4,1,1,1,10000,0,0,1000,1520,0.25
5,3,1,1,2500,15,4,500,1520,0.20
6,1,1,1,10000,0,0,1000,1520,0.25
7,1,0,0,10000,0,1,1000,1520,0.25
8,5,1,0,350,100,0,300,1520,0.25
9,1,1,0,10000,0,0,1000,1520,0.40


In [94]:
trains

,mpoezda_min,mpoezda_max,poezd_gamma,meankver_force,meankgor_force,meankprd_force,meankram_force,rmskver_force,rmskgor_force,rmskprd_force,rmskram_force
0,0,5000,0.15,2,2,2,2,2,2,2,2
1,0,5000,0.15,2,2,2,2,2,2,2,2
2,0,5000,0.15,2,2,2,2,2,2,2,2
3,0,5000,0.10,2,2,2,2,2,2,2,2
4,0,5000,0.10,2,2,2,2,2,2,2,2
5,0,5000,0.10,2,2,2,2,2,2,2,2
6,0,5000,0.10,2,2,2,2,2,2,2,2
7,0,5000,0.15,2,2,2,2,2,2,2,2


In [11]:
pod_sos

,vag_type,p_os,gamma,gamma_20,gamma_40,gamma_60,gamma_80,gamma_100,gamma_120,gamma_140,meankver_force,meankgor_force,meankprd_force,meankram_force,rmskver_force,rmskgor_force,rmskprd_force,rmskram_force
0,1,12,0.1,0.00,0.2,0.2,0.2,0.25,0.10,0.05,1,1,1,1,1,1,1,1
1,2,10,0.2,0.05,0.2,0.2,0.2,0.20,0.15,0.00,1,1,1,1,1,1,1,1
2,3,25,0.4,0.20,0.2,0.4,0.2,0.00,0.00,0.00,1,1,1,1,1,1,1,1
3,4,27,0.3,0.20,0.2,0.5,0.1,0.00,0.00,0.00,1,1,1,1,1,1,1,1
4,5,27,0.0,0.20,0.2,0.5,0.1,0.00,0.00,0.00,1,1,1,1,1,1,1,1


In [6]:
input_val = [2, 2, 250, 140, 80, 27.5, 1520, 0.25]
print ()
force = vagon_1 (input_val, 'return')
print (force)


[[61.8, 9.7, 13.350000000000001, 6.0], [79.1, 9.9, 19.3, 7.7], [44.5, 9.4, 7.4, 3.6]]


In [7]:
input_val = [2, 2, 250, 140, 80, 27.5, 1520, 0.25]
print ()
force = vagon_2 (input_val, 'return')
print (force)


[[137.4, 11.5, 38.2, 6.2], [185.4, 12.0, 53.7, 7.6], [89.5, 11.0, 22.6, 4.5]]


In [4]:
input_val = [2, 2, 250, 140, 80, 27.5, 1520, 0.25]
print ()
force = vagon_3 (input_val, 'return')
print (force)


[[109.35, 10.8, 22.25, 8.1], [135.6, 11.1, 32.7, 11.0], [83.1, 10.5, 11.8, 3.4]]


In [5]:
input_val = [2, 2, 250, 140, 80, 27.5, 1520, 0.25]
print ()
force = vagon_4 (input_val, 'return')
print (force)


[[120.5, 16.1, 25.65, 10.2], [155.3, 16.2, 38.5, 13.8], [85.7, 15.9, 12.8, 4.4]]


In [79]:
print (input_val)

[0. 0. 0. 0. 0. 0. 0. 0.]


In [80]:
len(line.index)

13

In [ ]:
vsp_type	vsp_cstr	vsp_cnd	rad_m	h_mm	i_prm	l_m	Shkol_mm	f_tr
VSP_type = input_val[0] 
    Condition = input_val[1] 
    Radius = input_val[2]
    h = input_val[3]
    V = input_val[4]
    axle_load = input_val[5]
    Sh_Kol = input_val[6] 
    mu_fr = input_val[7] 
    vag_type	p_os	gamma	gamma_20	gamma_40	gamma_60	
    gamma_80	gamma_100	gamma_120	gamma_140	
    meankver_force	meankgor_force	meankprd_force	meankram_force	
    rmskver_force	rmskgor_force	rmskprd_force	rmskram_force


In [95]:
# бЛОК вычисления сил для каждого элемента плана с полным учетом вагонопотока на линии
force_line = np.zeros((len(line.index),12))
for index_line in range(len(line.index)):
    #print (index)
    input_val = np.array([0, 0, 0,0,0,0,0,0.00])
    input_val[0]= line['vsp_cstr'].loc[line['vsp_cstr'].index == index_line].values
    input_val[1]= line['vsp_cnd'].loc[line['vsp_cnd'].index == index_line].values
    input_val[2]= line['rad_m'].loc[line['rad_m'].index == index_line].values
    input_val[3]= line['h_mm'].loc[line['h_mm'].index == index_line].values
    input_val[6]= line['Shkol_mm'].loc[line['Shkol_mm'].index == index_line].values
    input_val[7]= (line['f_tr'].loc[line['f_tr'].index == index_line].values)
    force_index = np.array([0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])
    for index_vagon in range(len(pod_sos.index)):
        gamma = pod_sos['gamma'].loc[pod_sos['gamma'].index == index_vagon].values
        p_os = pod_sos['p_os'].loc[pod_sos['p_os'].index == index_vagon].values
        vag_type = pod_sos['vag_type'].loc[pod_sos['vag_type'].index == index_vagon].values  
        meankver_force = pod_sos['meankver_force'].loc[pod_sos['meankver_force'].index == index_vagon].values  
        meankgor_force = pod_sos['meankgor_force'].loc[pod_sos['meankgor_force'].index == index_vagon].values  
        meankprd_force = pod_sos['meankprd_force'].loc[pod_sos['meankprd_force'].index == index_vagon].values  
        meankram_force = pod_sos['meankram_force'].loc[pod_sos['meankram_force'].index == index_vagon].values  
        rmskver_force = pod_sos['rmskver_force'].loc[pod_sos['rmskver_force'].index == index_vagon].values  
        rmskgor_force = pod_sos['rmskgor_force'].loc[pod_sos['rmskgor_force'].index == index_vagon].values  
        rmskprd_force = pod_sos['rmskprd_force'].loc[pod_sos['rmskprd_force'].index == index_vagon].values  
        rmskram_force = pod_sos['rmskram_force'].loc[pod_sos['rmskram_force'].index == index_vagon].values  
        force_v = np.array([0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])
        for v in (10,30,50,70,90,110,130):
            gamma_v_name = 'gamma_' + str(10+v)
            gamma_v = pod_sos[gamma_v_name].loc[pod_sos[gamma_v_name].index == index_vagon].values 
            input_val[4] = v
            input_val[5] = p_os
            #print (input_val)
            #print (vag_type, gamma_v)
            if vag_type == 1:
                force = vagon_1 (input_val, 'return')*gamma_v
                for i in [0,1,2]:
                    force[i] = force[i]*meankver_force
                for i in [3,4,5]:
                    force[i] = force[i]*rmskver_force
                for i in [6,7,8]:
                    force[i] = force[i]*meankgor_force
                for i in [9,10,11]:
                    force[i] = force[i]*rmskgor_force
                #print (force)
            elif vag_type == 2:
                force = vagon_2 (input_val, 'return')*gamma_v
                #print (force)
                for i in [0,1,2]:
                    force[i] = force[i]*meankver_force
                for i in [3,4,5]:
                    force[i] = force[i]*rmskver_force
                for i in [6,7,8]:
                    force[i] = force[i]*meankgor_force
                for i in [9,10,11]:
                    force[i] = force[i]*rmskgor_force
            elif vag_type == 3:
                force = vagon_3 (input_val, 'return')*gamma_v
                #print (force)
                for i in [0,1,2]:
                    force[i] = force[i]*meankver_force
                for i in [3,4,5]:
                    force[i] = force[i]*rmskver_force
                for i in [6,7,8]:
                    force[i] = force[i]*meankgor_force
                for i in [9,10,11]:
                    force[i] = force[i]*rmskgor_force
            elif vag_type == 4:
                force = vagon_4 (input_val, 'return')*gamma_v
                #print (force)
                for i in [0,1,2]:
                    force[i] = force[i]*meankver_force
                for i in [3,4,5]:
                    force[i] = force[i]*rmskver_force
                for i in [6,7,8]:
                    force[i] = force[i]*meankgor_force
                for i in [9,10,11]:
                    force[i] = force[i]*rmskgor_force
            #else:
                #print ("Номер типа подвижного состава указан неверно или отстуствует в базе")
            force_v = force_v + force
        force_index = force_index +  force_v*gamma 
    for i in range(0,12):
        force_line[index_line, i] = force_index[i]
print (force_line)



[[ 84.8355   93.382    89.10825  10.351    11.3565   10.897    17.067
   31.0465   24.05325   7.2145    8.5125    7.904  ]
 [ 84.8305   93.3625   89.094    10.338    11.3625   10.89     17.0655
   31.0465   24.0525    7.2145    8.512     7.904  ]
 [ 88.0345   94.9475   91.487     4.044     4.594     4.3395   10.6345
   20.9845   15.806     4.6985    4.4475    4.606  ]
 [ 84.833    93.3575   89.09125  10.368    11.3655   10.909    17.0655
   31.0465   24.0525    7.2155    8.512     7.903  ]
 [ 84.8355   93.382    89.10825  10.351    11.3565   10.897    17.067
   31.0465   24.05325   7.2145    8.5125    7.904  ]
 [114.248    84.9755   99.61525   2.305     2.4665    2.42      5.7435
   13.4615    9.5965    5.821     6.512     6.2025 ]
 [ 84.8355   93.382    89.10825  10.351    11.3565   10.897    17.067
   31.0465   24.05325   7.2145    8.5125    7.904  ]
 [ 84.8355   93.382    89.10825  10.351    11.3565   10.897    17.067
   31.0465   24.05325   7.2145    8.5125    7.904  ]
 [ 97.383   

In [96]:
# бЛОК вычисления сил для каждого элемента плана с полным учетом поездопотока на линии
force_poezd = np.zeros((len(line.index),12))
for index_line in range(len(line.index)):
    force_t = np.array([0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])
    
    for index_poezd in range(len(trains.index)):
        gamma_poezd = trains['poezd_gamma'].loc[trains['poezd_gamma'].index == index_poezd].values
        meankver_force = trains['meankver_force'].loc[trains['meankver_force'].index == index_poezd].values  
        meankgor_force = trains['meankgor_force'].loc[trains['meankgor_force'].index == index_poezd].values  
        meankprd_force = trains['meankprd_force'].loc[trains['meankprd_force'].index == index_poezd].values  
        meankram_force = trains['meankram_force'].loc[trains['meankram_force'].index == index_poezd].values  
        rmskver_force = trains['rmskver_force'].loc[trains['rmskver_force'].index == index_poezd].values  
        rmskgor_force = trains['rmskgor_force'].loc[trains['rmskgor_force'].index == index_poezd].values  
        rmskprd_force = trains['rmskprd_force'].loc[trains['rmskprd_force'].index == index_poezd].values  
        rmskram_force = trains['rmskram_force'].loc[trains['rmskram_force'].index == index_poezd].values
        for i in [0,1,2]:
            force[i] = force_line[index_line][i]*meankver_force*gamma_poezd
        for i in [3,4,5]:
            force[i] = force_line[index_line][i]*rmskver_force*gamma_poezd
        for i in [6,7,8]:
            force[i] = force_line[index_line][i]*meankgor_force*gamma_poezd
        for i in [9,10,11]:
            force[i] = force_line[index_line][i]*rmskgor_force*gamma_poezd 
        #print(force)
        force_t = force_t + force 
    #print (force_t)
    for i in range(0,12):
        force_poezd[index_line, i] = force_t[i]
print (force_poezd)


    

    

[[169.671  186.764  178.2165  20.702   22.713   21.794   34.134   62.093
   48.1065  14.429   17.025   15.808 ]
 [169.661  186.725  178.188   20.676   22.725   21.78    34.131   62.093
   48.105   14.429   17.024   15.808 ]
 [176.069  189.895  182.974    8.088    9.188    8.679   21.269   41.969
   31.612    9.397    8.895    9.212 ]
 [169.666  186.715  178.1825  20.736   22.731   21.818   34.131   62.093
   48.105   14.431   17.024   15.806 ]
 [169.671  186.764  178.2165  20.702   22.713   21.794   34.134   62.093
   48.1065  14.429   17.025   15.808 ]
 [228.496  169.951  199.2305   4.61     4.933    4.84    11.487   26.923
   19.193   11.642   13.024   12.405 ]
 [169.671  186.764  178.2165  20.702   22.713   21.794   34.134   62.093
   48.1065  14.429   17.025   15.808 ]
 [169.671  186.764  178.2165  20.702   22.713   21.794   34.134   62.093
   48.1065  14.429   17.025   15.808 ]
 [194.766  186.966  190.867   13.896   14.01    14.002   35.627   32.026
   33.8225   8.474   11.19    1

In [82]:
In [5]: df1.merge(df2) # by default, it does an inner join on the common column(s)
Out[5]: 
   x  y  z
0  2  b  4
1  3  c  5

df2 = df1.copy()
with pd.ExcelWriter('output.xlsx') as writer:  
    df1.to_excel(writer, sheet_name='Sheet_name_1')
    df2.to_excel(writer, sheet_name='Sheet_name_2')
    
#ExcelWriter также можно использовать для добавления в существующий файл Excel:

with pd.ExcelWriter('output.xlsx',
                    mode='a') as writer:  
    df.to_excel(writer, sheet_name='Sheet_name_3')
    
b[:2]  # Первые две строки
array([[ 0,  1,  2,  3],
       [10, 11, 12, 13]])
>>> b[1:3, : : ]  # Вторая и третья строки
array([[10, 11, 12, 13],
       [20, 21, 22, 23]])

SyntaxError: invalid syntax (<ipython-input-82-be34fe8908d3>, line 2)